In [1]:
import os
import json
import librosa
import numpy as np
import pandas as pd

# 🔹 소음 카테고리별 원천 데이터(WAV) 폴더
noise_categories = {
    "차량경적": "F:/noise_project/원천데이터/1.자동차/1.차량경적",
    "차량사이렌": "F:/noise_project/원천데이터/1.자동차/2.차량사이렌",
    "차량주행음": "F:/noise_project/원천데이터/1.자동차/3.차량주행음",
    "이륜차경적": "F:/noise_project/원천데이터/2.이륜자동차/4.이륜차경적",
    "이륜차주행음": "F:/noise_project/원천데이터/2.이륜자동차/5.이륜차주행음"
}

# 🔹 데시벨 임계값 설정
LOW_DB_THRESHOLD = 20  # 너무 조용한 파일 (무음 or 노이즈)
HIGH_DB_THRESHOLD = 110  # 너무 시끄러운 파일 (잘못된 녹음 or 극단적 노이즈)

# 🔹 SPL 기준값 (20μPa → 0dB 기준)
SPL_REFERENCE = 20e-6  

# 🔹 분석 결과 저장용 데이터프레임
results = []

for category, audio_folder in noise_categories.items():
    print(f"\n🔎 {category} 소음 분석 시작...")

    # 🔹 원천 데이터(WAV) 파일 리스트 가져오기
    wav_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
    total_wav_files = len(wav_files)

    # 🔹 분석 데이터 저장용 리스트
    noisy_files = []
    damaged_files = []
    rms_decibels = []

    for wav_file in wav_files:
        try:
            audio_path = os.path.join(audio_folder, wav_file)
            y, sr = librosa.load(audio_path, sr=None)

            if len(y) == 0:
                damaged_files.append(wav_file)
                continue

            # ✅ RMS(평균 제곱근) 계산
            rms = np.sqrt(np.mean(y ** 2))

            # ✅ SPL 기준 데시벨 변환 (이제 현실적인 값으로 변환됨!)
            spl_db = 20 * np.log10(rms / SPL_REFERENCE + 1e-6)

            rms_decibels.append(spl_db)

            # ✅ 노이즈 필터링 적용
            if spl_db < LOW_DB_THRESHOLD or spl_db > HIGH_DB_THRESHOLD:
                noisy_files.append(wav_file)

            # ✅ 결과 저장
            results.append({"Category": category, "File": wav_file, "SPL_dB": spl_db})

        except Exception as e:
            print(f"❌ 분석 불가 파일: {wav_file}, 오류: {e}")
            damaged_files.append(wav_file)

    # 🔹 분석 결과 출력 (VSCode 터미널)
    print(f"📂 총 WAV 파일 개수: {total_wav_files}")
    print(f"❌ 손상된 파일 개수: {len(damaged_files)}")
    print(f"❌ 노이즈(이상치) 파일 개수: {len(noisy_files)}")
    print(f"📊 SPL 분석 데시벨 분포 (상위 10개): {sorted(rms_decibels, reverse=True)[:10]}")
    print("-" * 50)

# 🔹 분석 데이터 DataFrame으로 변환
df_results = pd.DataFrame(results)

# 🔹 CSV 파일로 저장
output_path = "F:/noise_project/소음_분석_결과.csv"
df_results.to_csv(output_path, index=False)

print(f"✅ 모든 소음 데이터 분석 완료! 결과 저장: {output_path}")



🔎 차량경적 소음 분석 시작...
📂 총 WAV 파일 개수: 3189
❌ 손상된 파일 개수: 0
❌ 노이즈(이상치) 파일 개수: 0
📊 SPL 분석 데시벨 분포 (상위 10개): [85.69566871711486, 85.24554480325222, 85.1379433567634, 85.05844182602112, 85.0547930198685, 85.02763629665539, 85.00794863600217, 84.84574582752556, 84.74156273483857, 84.57880250893385]
--------------------------------------------------

🔎 차량사이렌 소음 분석 시작...
📂 총 WAV 파일 개수: 1990
❌ 손상된 파일 개수: 0
❌ 노이즈(이상치) 파일 개수: 0
📊 SPL 분석 데시벨 분포 (상위 10개): [87.13988257514431, 86.91338174611286, 86.36954706286531, 85.9499313342751, 85.20656272360267, 84.95442566094282, 84.67837416819077, 84.6561794839571, 84.00838604375514, 83.69676001096826]
--------------------------------------------------

🔎 차량주행음 소음 분석 시작...
📂 총 WAV 파일 개수: 1682
❌ 손상된 파일 개수: 0
❌ 노이즈(이상치) 파일 개수: 0
📊 SPL 분석 데시벨 분포 (상위 10개): [87.66475466557452, 87.64243391305601, 87.58160701408359, 86.34997612409333, 86.21108753087482, 85.81414550729461, 85.65766566327218, 85.112866486723, 85.04360638117143, 84.67880543003716]
--------------------------

In [2]:
import os
import json
import librosa
import numpy as np
import pandas as pd

# 🔹 소음 카테고리별 폴더 경로 설정
noise_categories = {
    "차량경적": ("F:/noise_project/라벨데이터/1.자동차/1.차량경적", "F:/noise_project/원천데이터/1.자동차/1.차량경적"),
    "차량사이렌": ("F:/noise_project/라벨데이터/1.자동차/2.차량사이렌", "F:/noise_project/원천데이터/1.자동차/2.차량사이렌"),
    "차량주행음": ("F:/noise_project/라벨데이터/1.자동차/3.차량주행음", "F:/noise_project/원천데이터/1.자동차/3.차량주행음"),
    "이륜차경적": ("F:/noise_project/라벨데이터/2.이륜자동차/4.이륜차경적", "F:/noise_project/원천데이터/2.이륜자동차/4.이륜차경적"),
    "이륜차주행음": ("F:/noise_project/라벨데이터/2.이륜자동차/5.이륜차주행음", "F:/noise_project/원천데이터/2.이륜자동차/5.이륜차주행음"),
}

# 🔹 SPL 기준값 (20μPa → 0dB 기준)
SPL_REFERENCE = 20e-6  

# 🔹 데시벨 차이 저장 리스트
comparison_results = []

for category, (json_folder, audio_folder) in noise_categories.items():
    print(f"\n🔎 {category} JSON vs WAV 데시벨 비교 시작...")

    # JSON 파일 리스트 가져오기
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    for json_file in json_files:
        try:
            # JSON 파일 로드
            with open(os.path.join(json_folder, json_file), "r", encoding="utf-8") as f:
                data = json.load(f)

            # JSON에 저장된 데시벨 값 가져오기
            json_decibel = data["annotations"][0].get("decibel", None) if "annotations" in data else None
            if json_decibel is None:
                continue  # 데시벨 정보 없으면 스킵

            # WAV 파일 경로 찾기 (JSON 파일명과 매칭)
            base_filename = json_file.replace(".json", "_1.wav")
            wav_path = os.path.join(audio_folder, base_filename)

            if not os.path.exists(wav_path):
                print(f"❌ WAV 파일 없음: {wav_path}")
                continue

            # WAV 파일 분석
            y, sr = librosa.load(wav_path, sr=None)
            if len(y) == 0:
                continue  # 손상된 파일 스킵

            # RMS(평균 제곱근) 계산
            rms = np.sqrt(np.mean(y ** 2))

            # SPL 기준 데시벨 변환
            wav_decibel = 20 * np.log10(rms / SPL_REFERENCE + 1e-6)

            # 데시벨 차이 계산
            decibel_difference = abs(wav_decibel - json_decibel)

            # 데이터 저장
            comparison_results.append({
                "Category": category,
                "File": base_filename,
                "JSON_dB": json_decibel,
                "WAV_dB": round(wav_decibel, 2),
                "ΔdB": round(decibel_difference, 2)
            })

        except Exception as e:
            print(f"❌ 오류 발생: {json_file} | {e}")

# 🔹 비교 결과 DataFrame 변환
df_comparison = pd.DataFrame(comparison_results)

# 🔹 결과 요약 출력 (VSCode에서 확인 가능)
print("\n📊 JSON vs WAV 데시벨 비교 결과 (상위 10개):")
print(df_comparison.head(10))

# 🔹 결과 CSV 저장
output_path = "F:/noise_project/데시벨_비교_결과.csv"
df_comparison.to_csv(output_path, index=False)

print(f"✅ 모든 JSON vs WAV 데시벨 비교 완료! 결과 저장: {output_path}")



🔎 차량경적 JSON vs WAV 데시벨 비교 시작...

🔎 차량사이렌 JSON vs WAV 데시벨 비교 시작...

🔎 차량주행음 JSON vs WAV 데시벨 비교 시작...

🔎 이륜차경적 JSON vs WAV 데시벨 비교 시작...

🔎 이륜차주행음 JSON vs WAV 데시벨 비교 시작...

📊 JSON vs WAV 데시벨 비교 결과 (상위 10개):
  Category               File  JSON_dB  WAV_dB    ΔdB
0     차량경적      1.자동차_1_1.wav       72   64.28   7.72
1     차량경적    1.자동차_100_1.wav       69   71.49   2.49
2     차량경적  1.자동차_10000_1.wav      116   78.58  37.42
3     차량경적  1.자동차_10001_1.wav       99   70.21  28.79
4     차량경적  1.자동차_10002_1.wav       76   75.08   0.92
5     차량경적  1.자동차_10004_1.wav      111   82.88  28.12
6     차량경적  1.자동차_10005_1.wav      121   80.18  40.82
7     차량경적  1.자동차_10006_1.wav      127   73.10  53.90
8     차량경적  1.자동차_10007_1.wav      105   80.47  24.53
9     차량경적  1.자동차_10010_1.wav      109   76.99  32.01
✅ 모든 JSON vs WAV 데시벨 비교 완료! 결과 저장: F:/noise_project/데시벨_비교_결과.csv
